In [1]:
!pip install "optimum[onnxruntime]==1.5.0" transformers evaluate mkl-include mkl --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install -U sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
from pathlib import Path


model_id="sentence-transformers/all-distilroberta-v1"
onnx_path = Path("onnx")

# load vanilla transformers and convert to onnx
model = ORTModelForFeatureExtraction.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.json',
 'onnx/merges.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

In [4]:
from transformers import Pipeline
import torch.nn.functional as F
import torch

# copied from the model card
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class SentenceEmbeddingPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        # we don't have any hyperameters to sanitize
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs):
        encoded_inputs = self.tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')
        return encoded_inputs

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        return {"outputs": outputs, "attention_mask": model_inputs["attention_mask"]}

    def postprocess(self, model_outputs):
        # Perform pooling
        sentence_embeddings = mean_pooling(model_outputs["outputs"], model_outputs['attention_mask'])
        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings


In [5]:
# init pipeline
vanilla_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer)

# run inference
pred = vanilla_emb("Could you assist me in finding my lost card?")

# print an excerpt from the sentence embedding
print(pred)
#     tensor([-0.0631,  0.0426,  0.0037,  0.0377,  0.0414])

print(len(pred[0]))


tensor([[ 2.2152e-03,  1.5510e-02,  1.0680e-02,  6.0858e-02,  1.1460e-02,
          1.1246e-02, -2.4895e-02,  6.8049e-02,  3.7286e-02, -8.8710e-03,
         -4.2174e-02,  3.8887e-02, -5.9415e-03, -1.6181e-02, -7.5151e-02,
         -4.9978e-02, -3.2811e-02,  4.9544e-02, -2.0334e-02,  8.7756e-03,
         -2.9489e-02, -4.5317e-02, -1.5661e-02,  1.4115e-02,  2.1258e-03,
         -5.8682e-02,  1.6960e-02, -2.9406e-02,  7.8740e-02,  1.7142e-02,
         -4.5779e-02,  1.7945e-02, -4.2303e-03, -2.3643e-02,  1.5234e-02,
         -3.4344e-02, -3.6330e-02,  4.8206e-02, -8.0538e-02, -9.3847e-03,
          4.0213e-02, -6.1752e-02,  1.2847e-02,  5.9283e-03, -3.8425e-03,
         -1.9850e-02, -1.3383e-02,  2.5059e-02,  4.6504e-02, -3.9689e-02,
         -1.2741e-02, -6.5160e-02,  9.5182e-03,  4.2189e-02, -9.8538e-03,
          3.6840e-02, -5.2671e-02, -6.4230e-02,  7.1371e-02,  2.0636e-02,
         -4.7696e-02, -7.0683e-03, -4.5537e-02, -4.3607e-02,  3.5107e-02,
         -6.5010e-03,  1.0711e-02,  3.

In [6]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.optimize(
    save_dir=onnx_path,
    optimization_config=optimization_config,
)


PosixPath('onnx')

In [7]:
from optimum.onnxruntime import ORTModelForFeatureExtraction

# load optimized model
model = ORTModelForFeatureExtraction.from_pretrained(onnx_path, file_name="model_optimized.onnx")

# create optimized pipeline
optimized_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer)
pred = optimized_emb("Could you assist me in finding my lost card?")
print(pred)
#  tensor([-0.0631,  0.0426,  0.0037,  0.0377,  0.0414])


tensor([[ 2.2152e-03,  1.5510e-02,  1.0680e-02,  6.0858e-02,  1.1460e-02,
          1.1246e-02, -2.4895e-02,  6.8049e-02,  3.7286e-02, -8.8710e-03,
         -4.2174e-02,  3.8887e-02, -5.9415e-03, -1.6181e-02, -7.5151e-02,
         -4.9978e-02, -3.2811e-02,  4.9544e-02, -2.0334e-02,  8.7756e-03,
         -2.9489e-02, -4.5317e-02, -1.5661e-02,  1.4115e-02,  2.1258e-03,
         -5.8682e-02,  1.6960e-02, -2.9406e-02,  7.8740e-02,  1.7142e-02,
         -4.5779e-02,  1.7945e-02, -4.2303e-03, -2.3643e-02,  1.5234e-02,
         -3.4344e-02, -3.6330e-02,  4.8206e-02, -8.0538e-02, -9.3847e-03,
          4.0213e-02, -6.1752e-02,  1.2847e-02,  5.9283e-03, -3.8426e-03,
         -1.9850e-02, -1.3383e-02,  2.5059e-02,  4.6504e-02, -3.9689e-02,
         -1.2740e-02, -6.5160e-02,  9.5182e-03,  4.2189e-02, -9.8538e-03,
          3.6840e-02, -5.2671e-02, -6.4230e-02,  7.1371e-02,  2.0636e-02,
         -4.7696e-02, -7.0683e-03, -4.5537e-02, -4.3607e-02,  3.5107e-02,
         -6.5011e-03,  1.0711e-02,  3.

In [8]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
dynamic_quantizer = ORTQuantizer.from_pretrained(model)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
model_quantized_path = dynamic_quantizer.quantize(
    save_dir=onnx_path,
    quantization_config=dqconfig,
)


In [9]:
import os

# get model file size
size = os.path.getsize(onnx_path / "model_optimized.onnx")/(1024*1024)
quantized_model = os.path.getsize(onnx_path / "model_optimized_quantized.onnx")/(1024*1024)

print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")


Model file size: 313.28 MB
Quantized Model file size: 220.49 MB


In [10]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer

model = ORTModelForFeatureExtraction.from_pretrained(onnx_path,file_name="model_optimized_quantized.onnx")
tokenizer = AutoTokenizer.from_pretrained(onnx_path)

q8_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer)

pred = q8_emb("Could you assist me in finding my lost card?")
print(pred[0][:5])
# tensor([-0.0567,  0.0111, -0.0110,  0.0450,  0.0447])


tensor([0.0039, 0.0157, 0.0152, 0.0634, 0.0117])


In [11]:
from datasets import load_dataset
from evaluate import load

eval_dataset = load_dataset("glue","stsb",split="validation")
metric = load('glue', 'stsb')

# creating a subset for faster evaluation
# COMMENT IN to run evaluation on a subset of the dataset
# eval_dataset = eval_dataset.select(range(200))


Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [12]:
def compute_sentence_similarity(sentence_1, sentence_2,pipeline):
    embedding_1 = pipeline(sentence_1)
    embedding_2 = pipeline(sentence_2)
    # compute cosine similarity between two sentences
    return torch.nn.functional.cosine_similarity(embedding_1, embedding_2, dim=1)


def evaluate_stsb(example):
  default = compute_sentence_similarity(example["sentence1"], example["sentence2"], vanilla_emb)
  quantized = compute_sentence_similarity(example["sentence1"], example["sentence2"], q8_emb)
  return {
      'reference': (example["label"] - 1) / (5 - 1), # rescale to [0,1]
      'default': float(default),
      'quantized': float(quantized),
      }

# run evaluation
result = eval_dataset.map(evaluate_stsb)

# compute metrics
default_acc = metric.compute(predictions=result["default"], references=result["reference"])
quantized = metric.compute(predictions=result["quantized"], references=result["reference"])

print(f"vanilla model: pearson={default_acc['pearson']}%")
print(f"quantized model: pearson={quantized['pearson']}%")
print(f"The quantized model achieves {round(quantized['pearson']/default_acc['pearson'],2)*100:.2f}% accuracy of the fp32 model")


  0%|          | 0/1500 [00:00<?, ?ex/s]

vanilla model: pearson=0.8829340574695262%
quantized model: pearson=0.8817374489605803%
The quantized model achieves 100.00% accuracy of the fp32 model


In [13]:
from time import perf_counter
import numpy as np

payload="Hello, my name is Philipp and I live in Nuremberg, Germany. Currently I am working as a Technical Lead at Hugging Face to democratize artificial intelligence through open source and open science. In the past I designed and implemented cloud-native machine learning architectures for fin-tech and insurance companies. I found my passion for cloud concepts and machine learning 5 years ago. Since then I never stopped learning. Currently, I am focusing myself in the area NLP and how to leverage models like BERT, Roberta, T5, ViT, and GPT2 to generate business value. I cannot wait to see what is next for me"
print(f'Payload sequence length: {len(tokenizer(payload)["input_ids"])}')

def measure_latency(pipe):
    latencies = []
    # warm up
    for _ in range(10):
        _ = pipe(payload)
    # Timed run
    for _ in range(100):
        start_time = perf_counter()
        _ =  pipe(payload)
        latency = perf_counter() - start_time
        latencies.append(latency)
    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    time_p95_ms = 1000 * np.percentile(latencies,95)
    return f"P95 latency (ms) - {time_p95_ms}; Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f};", time_p95_ms


vanilla_model=measure_latency(vanilla_emb)
quantized_model=measure_latency(q8_emb)

print(f"Vanilla model: {vanilla_model[0]}")
print(f"Quantized model: {quantized_model[0]}")
print(f"Improvement through quantization: {round(vanilla_model[1]/quantized_model[1],2)}x")


Payload sequence length: 132
Vanilla model: P95 latency (ms) - 330.9908687999837; Average latency (ms) - 235.80 +\- 57.49;
Quantized model: P95 latency (ms) - 249.34936809997907; Average latency (ms) - 179.10 +\- 42.55;
Improvement through quantization: 1.33x
